In [ ]:
import os 
import sys
import scanpy as sc

# Append ghostcoder folder to path 
sys.path.append(os.path.abspath('..'))

import ghostcoder

from ghostcoder import GhostCoder
from ghostcoder.utils import *
from ghostcoder.graph import create_ghostcoder_agent, create_coder_agent, create_crawler_agent, create_retriever_agent

ImportError: cannot import name 'create_rag_agent' from 'ghostcoder.graph' (/home/qixin/BIA-Ghostcoder/ghostcoder/graph/__init__.py)

Here we will test and illustrate each subgraph in Ghostcoder, as an important component of BIA (bioinformatics agnet), mainly functions to complete the generation and execution of bioinformatics analysis codes. It contains five subgraphs. They are filemanager, retriever, coder and webcrawler executor respectively.


### File system and omics data preparation

In [2]:
# Set task_id as "Test"
ghostcoder.config.TASK_ID = "Test"

"""
Initial file system status as follow: 
├───Test  
└───data  
  └─pbmc3k_raw.h5ad  
Where we set 
  """


NameError: name 'ghostcoder' is not defined

### Model preparation 

In [ ]:
# Use Openai API for example, you can use other LLM provider as well
from langchain_openai import ChatOpenAI

# I will recommend use Openai API, I haven't tested any APIs from other suppliers yet
def call_chatllm_openai(api_key, api_base, model_name):
    llm = ChatOpenAI(
        openai_api_key = api_key,
        openai_api_base=api_base,
        model = model_name)
    return llm

# Setup up api keys 
# openai_api_key = ""
# openai_api_base = ""
# openai_chat_model = "" 
# openai_code_model = ""

# Here, I recommend using an LLM with stronger coding capabilities as the code model, which is set to perform code-related work in all Ghostcoder graphs. Meanwhile, it is better to use LLM with stronger reasoning ability as the chat model.
chat_model = call_chatllm_openai(openai_api_key, openai_api_base, openai_chat_model)
code_model = call_chatllm_openai(openai_api_key, openai_api_base, openai_code_model)

In [ ]:
# Initial Tavily
tavily_api = ""
os.environ["TAVILY_API_KEY"] = tavily_api

In [ ]:
# Data 
adata = sc.datasets.pbmc3k()

In [ ]:
# Initial graph
agent = GhostCoder(
    chat_model = chat_model, 
    code_model = code_model, 
    )

# Draw graph
agent.draw_graph()

In [ ]:
# Test agent
task = "Quality control of the data. wherein the genes are targeted for labeling, the mitochondrial genes (e.g., beginning with “MT-”), the ribosomal genes (e.g., beginning with “RPS” or “RPL”), and the hemoglobin genes (using regular expression matching, e.g., ^HB[^P]); next, common quality control metrics for each cell, including total counts, number of genes detected, and the proportion of total counts represented by a specific group of genes (e.g., mitochondrial genes), were calculated using Scanpy's calculate_qc_metrics() function, and a The log1p transformation is applied to these metrics to optimize the data distribution. Subsequently, the QC metrics of each cell are visualized by violin plots and scatter plots to assess the overall quality of the data. Finally, a threshold is set based on the visualization results to exclude the cells with fewer than 100 genes and genes occurring in fewer than 3 cells to ensure the quality of the data for downstream analyses."

In [ ]:
# Run Ghost coder
agent.Run(
    task, 
    input_wrap = input_variable_wrapper([adata])
    )